# Introducción a NLTK

In [1]:
# Cargamos librería
import nltk

# Descargamos dataset
nltk.download('cess_esp')

[nltk_data] Downloading package cess_esp to /home/missael/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


True

En este caso, descargamos _cess esp_, que es un corpus en español con titulares de noticias 👀.
## Expresiones regulares

Primero analizaremos los datos usando **expresiones regulares**.

In [2]:
# Libreria para regular expressions
import re

# Cargamos el corpus
corpus = nltk.corpus.cess_esp.sents()
print(corpus[0])
print(len(corpus))

['El', 'grupo', 'estatal', 'Electricité_de_France', '-Fpa-', 'EDF', '-Fpt-', 'anunció', 'hoy', ',', 'jueves', ',', 'la', 'compra', 'del', '51_por_ciento', 'de', 'la', 'empresa', 'mexicana', 'Electricidad_Águila_de_Altamira', '-Fpa-', 'EAA', '-Fpt-', ',', 'creada', 'por', 'el', 'japonés', 'Mitsubishi_Corporation', 'para', 'poner_en_marcha', 'una', 'central', 'de', 'gas', 'de', '495', 'megavatios', '.']
6030


Podemos ver un ejemplo de titular y vemos que tenemos $6030$ titulares, además de que ya está tokenizado todo. Ahora vamos a hacer _flatten_ a los datos, es decir, en lugar de tener una lista de listas, vamos a pasar todos los tokens en una sola lista. 

In [3]:
flatten = [word for list in corpus for word in list]
print(len(flatten))

192686


¡Tenemos muchísimas palabras!🤯

## Estructura de la función re.search()

Esta función determina si el patrón de la expresión regular aplica en una cadena de texto, re.searhc(p, s)

In [4]:
arr = [word for word in flatten if re.search('es',word)]
print(arr[:7])

['estatal', 'jueves', 'empresa', 'centrales', 'francesa', 'japonesa', 'millones']


Podemos ver que todos los elementos tienen dentro el texto _es_.🤯 Este _es_ es un metacaracter que define un patrón de búsqueda muy básico. Podemos crear patrónes de busqueda mucho más complejos.
- El caracter $ sirve para reemplazar cualquier caracter.
- ^ sirve para indicar que está al inicio de la cadena de texto.

In [5]:
arr = [word for word in flatten if re.search('es$', word)]
print(arr[:5])

arr = [word for word in flatten if re.search('^es', word)]
print(arr[:5])

['jueves', 'centrales', 'millones', 'millones', 'dólares']
['estatal', 'es', 'esta', 'esta', 'eso']


Ahora veremos que es el **rango**. La notación es [a-z], [ghi]. Cualquier letra en ese rango puede estar en dicho lugar

In [6]:
arr = [word for word in flatten if re.search('^[ghi]', word)]
print(arr[:7])

['grupo', 'hoy', 'gas', 'gas', 'intervendrá', 'invertir', 'gas']


Por último, tenemos clausuras, y hay dos tipos:
- *: Una cadena de texto se puede repetir 0 o más veces.
- +: La cadena se puede repetir 1 o más veces.

In [7]:
arr = [word for word in flatten if re.search('^(no)+', word)]
print(arr[:10])

['norte', 'no', 'no', 'noche', 'no', 'no', 'notificación', 'no', 'no', 'no']


## Tokenización con expresiones regulares

Python no interpreta el texto _raw_ por defecto, si no que admite cosas espciales como tabs con `\t` o enters con `\n`. Podemos indicarle que lea solo raw:

In [8]:
print('Esta es \n una prueba')
print(r'Esta es \n otra prueba')

Esta es 
 una prueba
Esta es \n otra prueba


**Tokenización**: Es el proceso mediante el cual se sub-divide una cadena de texto en unidades linguísticas minimas (palabras)

In [9]:
texto = """ Cuando sea el rey del mundo (imaginaba él en su cabeza) no tendré que  preocuparme por estas bobadas.
            Era solo un niño de 7 años, pero pensaba que podría ser cualquier cosa que su imaginación le permitiera 
            visualizar en su cabeza ..."""

**Caso 1**: Tokenizar por espacios vacios:

In [10]:
print(re.split(r' ', texto))

['', 'Cuando', 'sea', 'el', 'rey', 'del', 'mundo', '(imaginaba', 'él', 'en', 'su', 'cabeza)', 'no', 'tendré', 'que', '', 'preocuparme', 'por', 'estas', 'bobadas.\n', '', '', '', '', '', '', '', '', '', '', '', 'Era', 'solo', 'un', 'niño', 'de', '7', 'años,', 'pero', 'pensaba', 'que', 'podría', 'ser', 'cualquier', 'cosa', 'que', 'su', 'imaginación', 'le', 'permitiera', '\n', '', '', '', '', '', '', '', '', '', '', '', 'visualizar', 'en', 'su', 'cabeza', '...']


Aqui hay ruido, caracteres que no son parte de las palabras y queremos remover.

**Caso 2**: Tokenizador usando expresiones regulares

In [11]:
print(re.split(r'[ \W\t\n]+', texto))

['', 'Cuando', 'sea', 'el', 'rey', 'del', 'mundo', 'imaginaba', 'él', 'en', 'su', 'cabeza', 'no', 'tendré', 'que', 'preocuparme', 'por', 'estas', 'bobadas', 'Era', 'solo', 'un', 'niño', 'de', '7', 'años', 'pero', 'pensaba', 'que', 'podría', 'ser', 'cualquier', 'cosa', 'que', 'su', 'imaginación', 'le', 'permitiera', 'visualizar', 'en', 'su', 'cabeza', '']


### Tokenizador de NLTK

Si usamos lo anterior, nos ignora abreviaciones, precios, como esto: 

In [12]:
texto = 'En los E.U. esa postal vale $15.50 ...'
print(re.split('[ \W\t\n]+', texto))

['En', 'los', 'E', 'U', 'esa', 'postal', 'vale', '15', '50', '']


Para poder considerar esto, debemos usar expresiones regulares mucho más complejas, como esta: 

In [13]:
pattern = r'''(?x)                  # Flag para iniciar el modo verbose
              (?:[A-Z]\.)+          # Hace match con abreviaciones como U.S.A.
              | \w+(?:-\w+)*        # Hace match con palabras que pueden tener un guión interno
              | \$?\d+(?:\.\d+)?%?  # Hace match con dinero o porcentajes como $15.5 o 100%
              | \.\.\.              # Hace match con puntos suspensivos
              | [][.,;"'?():-_`]    # Hace match con signos de puntuación
            '''

# Usamos NLTK
nltk.regexp_tokenize(texto, pattern)

['En', 'los', 'E.U.', 'esa', 'postal', 'vale', '$15.50', '...']